In [1]:
import os
import scipy.io
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from tqdm import tqdm
from typing import Any

In [2]:
train_path = "CoordinateData\\train"
validation_path = "CoordinateData\\validation"

In [3]:
def segment_data(data: np.ndarray, window_size: int = 180, overlap_ratio: float = 0.75, by_type: bool = True, min_frame: int = 12) -> np.ndarray:
    """
    Segment the input data into smaller windows based on the given parameters.

    Args:
        data (np.ndarray): Input data with shape (num_samples, num_features).
        window_size (int, optional): The length of each window. Defaults to 180.
        overlap_ratio (float, optional): The ratio of overlap between consecutive windows. Defaults to 0.75.
        by_type (bool, optional): Whether to segment the data by type (assuming the 71st feature is the type). Defaults to True.
        min_frame (int, optional): The minimum number of frames required to create a new instance. Defaults to 12.

    Returns:
        np.ndarray: The segmented data with shape (num_windows, window_size, num_features).
    """

    # Check the input constraints
    assert data.shape[0] > 0
    assert window_size > 0
    assert 0 <= overlap_ratio < 1
    assert 0 <= min_frame < window_size

    dim = data.shape[1]
    instances = []

    if not by_type:
        instances.append(data)
    else:
        assert data.shape[1] >= 71

        num_data = data.shape[0]
        left, right = 0, 1
        pre_type = -1
        cur_type = data[left, 70]

        # Segment the data by exercise type
        while right < num_data:
            if data[right, 70] == cur_type:
                right += 1
                continue

            if right - left <= min_frame:
                left = right
                cur_type = data[left, 70]
                right += 1
                continue

            new_instance = np.take(data, range(left, right), axis=0)
            if pre_type == new_instance[0, 70]:
                instances[-1] = np.vstack([instances[-1], new_instance])
            else:
                instances.append(new_instance)

            left = right
            pre_type = cur_type
            cur_type = data[left, 70]
            right += 1

        # Handle the remaining data
        new_instance = np.take(data, range(left, right), axis=0)
        last = instances[-1]
        if last[0, 70] == new_instance[0, 70]:
            instances[-1] = np.vstack([last, new_instance])
        else:
            instances.append(new_instance)

    # print(len(instances))

    step_size = int(window_size * (1 - overlap_ratio))
    windows = []

    # Create windows for each instance
    for instance in instances:
        if instance.shape[0] < window_size:
            instance = np.vstack([instance, np.zeros((window_size - instance.shape[0], dim))])
            windows.append(instance)
            continue

        if (instance.shape[0] - window_size) % step_size != 0:
            pad_size = step_size - (instance.shape[0] - window_size) % step_size
            instance = np.vstack([instance, np.zeros((pad_size, dim))])

        for i in range(0, instance.shape[0] - window_size + 1, step_size):
            windows.append(np.take(instance, range(i, i + window_size), axis=0))

    return np.array(windows)

In [4]:
def ensemble_predict(y_predicts: np.ndarray, confusion_matrices: np.ndarray, ensemble_method: int = 0) -> np.ndarray:
    """
    Perform ensemble prediction using different methods based on multiple models' predictions and their confusion matrices.

    Parameters:
    y_predicts (np.ndarray): A 3D array of shape (num_modalities, num_windows, 2) containing prediction probabilities for each modality.
    confusion_matrices (np.ndarray): A 3D array of shape (num_modalities, 2, 2) containing confusion matrices for each modality.
    ensemble_method (int): An integer (0-7) indicating the ensemble method to be used. Default is 0 (Simple Average).

    Returns:
    np.ndarray: A 1D array of shape (num_windows,) containing the ensemble predictions for each window.
    """

    # Check input assertions
    assert y_predicts.shape[0] > 1
    assert y_predicts.shape[2] == 2
    assert y_predicts.shape[0] == confusion_matrices.shape[0]
    assert 0 <= ensemble_method < 8

    num_modalities = y_predicts.shape[0]
    num_windows = y_predicts.shape[1]
    print(f'{num_windows} data are predicted using combined results of {num_modalities} modalities')

    # Perform ensemble prediction based on the selected method
    if ensemble_method == 0:                                            # Simple Average
        final_predict = np.argmax(np.mean(y_predicts, axis=0), axis=1)
    elif ensemble_method in [1, 2, 3, 4]:                               # Weighted Average (using different metrics)
        accuracies = []
        precisions = []
        recalls = []
        f1_scores = []

        # Calculate metrics for each confusion matrix
        for cm in confusion_matrices:
            TN, FN, FP, TP = cm.ravel()

            accuracy = (TP + TN) / (TP + TN + FP + FN)
            precision = TP / (TP + FP)
            recall = 0.0 if TP == 0.0 else TP / (TP + FN)
            f1_score = 0.0 if TP == 0.0 else 2 * (precision * recall) / (precision + recall)

            accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1_score)

        # Normalize the metrics
        accuracies = np.array(accuracies) / np.sum(accuracies)
        precisions = np.array(precisions) if np.sum(precisions) == 0 else np.array(precisions) / np.sum(precisions)
        recalls = np.array(recalls) if np.sum(recalls) == 0 else np.array(recalls) / np.sum(recalls)
        f1_scores = np.array(f1_scores) if np.sum(f1_scores) == 0 else np.array(f1_scores) / np.sum(f1_scores)

        metrics = {
                1: accuracies,
                2: precisions,
                3: recalls,
                4: f1_scores
        }
        # Choose the appropriate metric for weighted averaging
        metric = metrics[ensemble_method]

        # Perform weighted averaging
        y_predicts = metric[:, np.newaxis, np.newaxis] * y_predicts
        final_predict = np.argmax(np.sum(y_predicts, axis=0), axis=1)
    elif ensemble_method == 5:                                          # Maximum Rule
        final_predict = np.argmax(np.max(y_predicts, axis=0), axis=1)
    elif ensemble_method == 6:                                          # Product Rule
        final_predict = np.argmax(np.prod(y_predicts, axis=0), axis=1)
    else:                                                               # Voting
        final_predict, _ = mode(np.argmax(y_predicts, axis=2), axis=0)
        final_predict = final_predict.flatten()

    assert final_predict.shape[0] == y_predicts.shape[1]
    return final_predict

In [9]:
def fit(estimator: Any, parameters: dict, path: str = train_path, base_parameters: dict = None, seg: bool = False, downsampling: bool = False) -> [Any]:
    models = []
    estimator_class = type(estimator)

    for i, file in tqdm(enumerate(os.listdir(path))):
        mat = scipy.io.loadmat(os.path.join(path, file))
        data = mat['data']
        if downsampling and np.unique(data[:, 72:73]).size == 1:
            print("downsampling!!!")
            continue

        if 'base_estimator' in parameters:
            base_class = type(parameters['base_estimator'])
            parameters['base_estimator'] = base_class(**base_parameters) if base_parameters else base_class()

        if seg:
            data = segment_data(data)
            X_train = data[:, :, 0:70]
            X_train = X_train.reshape(X_train.shape[0], -1)
            y_train = data[:, :, 72:73]
            y_train = np.apply_along_axis(lambda x: mode(x)[0], 1, y_train[:, :, 0]).flatten()
        else:
            X_train = np.delete(data, range(70, 78), axis=1)
            y_train = data[:, 72]

        model = estimator_class(**parameters)
        model.fit(X_train, y_train)
        models.append((file, model))

    print("Downsampling:", downsampling)
    return models

In [6]:
def validate(models: list, path: str = validation_path, seg: bool = False) -> None:
    total_true_protective = 0
    total_protective = 0
    total_true_non_protective = 0
    total_non_protective = 0

    for file in os.listdir(path):
        mat = scipy.io.loadmat(os.path.join(path, file))
        data = mat['data']
        if seg:
            data = segment_data(data)
            X_validation = data[:, :, 0:70]
            X_validation = X_validation.reshape(X_validation.shape[0], -1)
            y_validation = data[:, :, 72:73]
            y_validation = np.apply_along_axis(lambda x: mode(x)[0], 1, y_validation[:, :, 0]).flatten()
            assert X_validation.shape[0] == y_validation.shape[0]
        else:
            X_validation = np.delete(data, range(70, 78), axis=1)
            y_validation = data[:, 72]

        predictions = []
        for model in models:
            predictions.append(model[1].predict(X_validation))

        y_pred, _ = mode(predictions, axis=0)
        y_pred = y_pred.flatten()

        protective = np.where(y_validation == 1)
        non_protective = np.where(y_validation == 0)

        total_true_protective += np.sum(y_validation[protective] == y_pred[protective])
        total_protective += len(y_validation[protective])

        total_true_non_protective += np.sum(y_validation[non_protective] == y_pred[non_protective])
        total_non_protective += len(y_validation[non_protective])

    print("Files are segmented:", seg)
    print("Accuracy(protective):", total_true_protective / total_protective, total_true_protective, total_protective)
    print("Accuracy(non-protective):", total_true_non_protective / total_non_protective, total_true_non_protective, total_non_protective)
    TN, FN = total_true_non_protective, total_non_protective - total_true_non_protective
    FP, TP = total_protective - total_true_protective, total_true_protective
    precision = TP / (TP + FP)
    recall = 0.0 if TP == 0.0 else TP / (TP + FN)
    f1_score = 0.0 if TP == 0.0 else 2 * (precision * recall) / (precision + recall)
    print("F1-score:", f1_score)
    print("Overall accuracy:", (total_true_protective + total_true_non_protective) / (total_protective + total_non_protective))

In [7]:
rfcs = fit(RandomForestClassifier(), {'random_state': 42})
validate(rfcs)
rfcs = fit(RandomForestClassifier(), {'random_state': 42}, downsampling=True)
validate(rfcs)

23it [03:27,  9.02s/it]


Downsampling: False


2it [00:00, 15.25it/s]

Files are segmented: False
Accuracy(protective): 0.0 0 10721
Accuracy(non-protective): 0.9999932302984761 147716 147717
F1-score: 0.0
Overall accuracy: 0.9323268407831454


23it [03:26,  8.99s/it]


Downsampling: True
Files are segmented: False
Accuracy(protective): 0.34381121164070516 3686 10721
Accuracy(non-protective): 0.5322271641043347 78619 147717
F1-score: 0.0882821387940842
Overall accuracy: 0.5194776505636274


In [8]:
abcs = fit(AdaBoostClassifier(), {'random_state': 42})
validate(abcs)
abcs = fit(AdaBoostClassifier(), {'random_state': 42}, downsampling=True)
validate(abcs)

23it [01:19,  3.46s/it]


Downsampling: False


4it [00:00, 34.45it/s]

Files are segmented: False
Accuracy(protective): 0.0 0 10721
Accuracy(non-protective): 1.0 147717 147717
F1-score: 0.0
Overall accuracy: 0.932333152400308


23it [01:18,  3.42s/it]


Downsampling: True
Files are segmented: False
Accuracy(protective): 0.06435966794142338 690 10721
Accuracy(non-protective): 0.8747672915101173 129218 147717
F1-score: 0.04613841524573721
Overall accuracy: 0.819929562352466


In [9]:
abcs_dtc = fit(AdaBoostClassifier(), {'base_estimator': DecisionTreeClassifier(), 'random_state': 42})
validate(abcs_dtc)
abcs_dtc = fit(AdaBoostClassifier(), {'base_estimator': DecisionTreeClassifier(), 'random_state': 42}, downsampling=True)
validate(abcs_dtc)

23it [00:14,  1.63it/s]


Downsampling: False


4it [00:00, 35.37it/s]

Files are segmented: False
Accuracy(protective): 0.0427198955321332 458 10721
Accuracy(non-protective): 0.9379286067277294 138548 147717
F1-score: 0.04501670925889523
Overall accuracy: 0.8773526552973403


23it [00:13,  1.66it/s]


Downsampling: True
Files are segmented: False
Accuracy(protective): 0.6675683238503871 7157 10721
Accuracy(non-protective): 0.18035838799867313 26642 147717
F1-score: 0.10301324908422273
Overall accuracy: 0.21332634847700677


In [10]:
rfcs_seg = fit(RandomForestClassifier(), {'random_state': 42}, seg=True)
validate(rfcs_seg, seg=True)
rfcs_seg = fit(RandomForestClassifier(), {'random_state': 42}, seg=True, downsampling=True)
validate(rfcs_seg, seg=True)

23it [00:13,  1.67it/s]


Downsampling: False


4it [00:00, 35.37it/s]

Files are segmented: True
Accuracy(protective): 0.0 0 171
Accuracy(non-protective): 0.9985174203113417 2694 2698
F1-score: 0.0
Overall accuracy: 0.9390031369815267


23it [00:12,  1.81it/s]


Downsampling: True
Files are segmented: True
Accuracy(protective): 0.39766081871345027 68 171
Accuracy(non-protective): 0.6942179392142328 1873 2698
F1-score: 0.12781954887218044
Overall accuracy: 0.67654234925061


In [11]:
abcs_seg = fit(AdaBoostClassifier(), {'random_state': 42}, seg=True)
validate(abcs_seg, seg=True)
abcs_seg = fit(AdaBoostClassifier(), {'random_state': 42}, seg=True, downsampling=True)
validate(abcs_seg, seg=True)

23it [03:13,  8.43s/it]


Downsampling: False


4it [00:00, 35.37it/s]

Files are segmented: True
Accuracy(protective): 0.0 0 171
Accuracy(non-protective): 1.0 2698 2698
F1-score: 0.0
Overall accuracy: 0.9403973509933775


23it [03:14,  8.44s/it]


Downsampling: True
Files are segmented: True
Accuracy(protective): 0.2982456140350877 51 171
Accuracy(non-protective): 0.9521868050407709 2569 2698
F1-score: 0.2905982905982906
Overall accuracy: 0.9132101777622865


In [12]:
abcs_dtc_seg = fit(AdaBoostClassifier(), {'base_estimator': DecisionTreeClassifier(), 'random_state': 42}, seg=True)
validate(abcs_dtc_seg, seg=True)
abcs_dtc_seg = fit(AdaBoostClassifier(), {'base_estimator': DecisionTreeClassifier(), 'random_state': 42}, seg=True, downsampling=True)
validate(abcs_dtc_seg, seg=True)

23it [00:20,  1.13it/s]


Downsampling: False


4it [00:00, 35.68it/s]

Files are segmented: True
Accuracy(protective): 0.0 0 171
Accuracy(non-protective): 1.0 2698 2698
F1-score: 0.0
Overall accuracy: 0.9403973509933775


23it [00:19,  1.16it/s]


Downsampling: True
Files are segmented: True
Accuracy(protective): 0.09941520467836257 17 171
Accuracy(non-protective): 0.9844329132690882 2656 2698
F1-score: 0.14782608695652175
Overall accuracy: 0.9316835134193099


In [109]:
def load_data(path: str = train_path, downsampling: bool = True):
    X_train_list = []
    y_train_list = []

    selected_data_list = []
    for file in os.listdir(path):
        mat = scipy.io.loadmat(os.path.join(path, file))
        if downsampling and np.unique(mat['data'][:, 72:73]).size == 1:
            continue
        else:
            selected_data_list.append(mat['data'])


    for i in range(len(selected_data_list)):
        processed_data = segment_data(selected_data_list[i], min_frame=0)

        X_segmented = processed_data[:,:,0:70]
        y_segmented = processed_data[:,:,72:73]

        y_segmented = np.apply_along_axis(lambda x: mode(x)[0], 1, y_segmented[:, :, 0])

        X_train_list.append(X_segmented)
        y_train_list.append(y_segmented)

    X_train_concat = np.concatenate(X_train_list, axis=0)
    y_train_concat = np.concatenate(y_train_list, axis=0)

    return X_train_concat, y_train_concat.flatten()

In [110]:
X_t, y_t = load_data()
print(X_t.shape, y_t.shape)

(5014, 180, 70) (5014,)


In [117]:
from sklearn.model_selection import train_test_split


train_data, test_data, train_labels, test_labels = train_test_split(X_t, y_t, test_size=0.2, random_state=42)
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

(4011, 180, 70)
(4011,)
(1003, 180, 70)
(1003,)


In [118]:
X_t_rf = np.take(train_data, range(0, 66), axis=2).reshape(train_data.shape[0], -1)
# std_scaler = StandardScaler()
# X_t_svm = std_scaler.fit_transform(np.take(train_data, range(66, 70), axis=2).reshape(train_data.shape[0], -1))
X_t_svm = np.take(train_data, range(66, 70), axis=2).reshape(train_data.shape[0], -1)

X_v_rf = np.take(test_data, range(0, 66), axis=2).reshape(test_data.shape[0], -1)
# X_v_svm = std_scaler.transform(np.take(test_data, range(66, 70), axis=2).reshape(test_data.shape[0], -1))
X_v_svm = np.take(test_data, range(66, 70), axis=2).reshape(test_data.shape[0], -1)

print(X_t_rf.shape, X_t_svm.shape, X_v_rf.shape, X_v_svm.shape)

(4011, 11880) (4011, 720) (1003, 11880) (1003, 720)


In [119]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_t_rf, train_labels)

RandomForestClassifier(random_state=42)

In [120]:
rf_confusion_matrix = confusion_matrix(test_labels, rf.predict(X_v_rf))

In [140]:
from sklearn.svm import SVC
from keras.utils import to_categorical


def hard_late_fusion(SVC_parameters: dict) -> None:
    svm = SVC(**SVC_parameters)
    svm.fit(X_t_svm, train_labels)

    svm_confusion_matrix = confusion_matrix(test_labels, svm.predict(X_v_svm))

    print("RF confusion matrix")
    print(rf_confusion_matrix)
    print("svm confusion matrix")
    print(svm_confusion_matrix)

    confusion_matrices = np.array([rf_confusion_matrix, svm_confusion_matrix])
    total_true_protective = 0
    total_protective = 0
    total_true_non_protective = 0
    total_non_protective = 0

    for file in os.listdir(validation_path):
        mat = scipy.io.loadmat(os.path.join(validation_path, file))
        data = segment_data(mat['data'], min_frame=0)
        X_validation_rf = np.take(data, range(0, 66), axis=2).reshape(data.shape[0], -1)
        X_validation_svm = np.take(data, range(66, 70), axis=2).reshape(data.shape[0], -1)
        y_validation = data[:, :, 72:73]
        y_validation = np.apply_along_axis(lambda x: mode(x)[0], 1, y_validation).flatten()

        rf_pred = to_categorical(rf.predict(X_validation_rf), 2)
        # svm_pred = to_categorical(svm.predict(std_scaler.transform(X_validation_svm)), 2)
        svm_pred = to_categorical(svm.predict(X_validation_svm), 2)
        print(rf_pred.shape, svm_pred.shape, y_validation.shape)

        protective = np.where(y_validation == 1)[0]
        non_protective = np.where(y_validation == 0)[0]

        y_pred = ensemble_predict(np.array([rf_pred, svm_pred]), confusion_matrices, ensemble_method=6)

        total_true_protective += np.sum(y_pred[protective] == y_validation[protective])
        total_protective += len(y_validation[protective])

        total_true_non_protective += np.sum(y_pred[non_protective] == y_validation[non_protective])
        total_non_protective += len(y_validation[non_protective])

    print("Accuracy(protective):", total_true_protective / total_protective, total_true_protective, total_protective)
    print("Accuracy(non-protective):", total_true_non_protective / total_non_protective, total_true_non_protective, total_non_protective)
    TN, FN = total_true_non_protective, total_non_protective - total_true_non_protective
    FP, TP = total_protective - total_true_protective, total_true_protective
    precision = TP / (TP + FP)
    recall = 0.0 if TP == 0.0 else TP / (TP + FN)
    f1_score = 0.0 if TP == 0.0 else 2 * (precision * recall) / (precision + recall)
    print("F1-score:", f1_score)
    print("Overall accuracy:", (total_true_protective + total_true_non_protective) / (total_protective + total_non_protective))

In [141]:
hard_late_fusion({'kernel': 'linear', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[664 143]
 [133  63]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [142]:
hard_late_fusion({'kernel': 'poly', 'degree': 2, 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[762  45]
 [151  45]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [143]:
hard_late_fusion({'kernel': 'poly', 'degree': 3, 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[767  40]
 [152  44]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [144]:
hard_late_fusion({'kernel': 'poly', 'degree': 4, 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[759  48]
 [146  50]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [145]:
hard_late_fusion({'kernel': 'rbf', 'gamma': 'scale', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[654 153]
 [ 86 110]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [146]:
hard_late_fusion({'kernel': 'rbf', 'gamma': 'auto', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[712  95]
 [138  58]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [147]:
hard_late_fusion({'kernel': 'rbf', 'C': 0.1, 'gamma': 'auto', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[762  45]
 [166  30]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [148]:
hard_late_fusion({'kernel': 'rbf', 'C': 10.0, 'gamma': 'auto', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[703 104]
 [135  61]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [149]:
hard_late_fusion({'kernel': 'rbf', 'C': 0.1, 'gamma': 'scale', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[657 150]
 [104  92]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi

In [150]:
hard_late_fusion({'kernel': 'rbf', 'C': 10.0, 'gamma': 'scale', 'random_state': 42, 'class_weight': 'balanced'})

RF confusion matrix
[[785  22]
 [ 26 170]]
svm confusion matrix
[[683 124]
 [108  88]]
(197, 2) (197, 2) (197,)
197 data are predicted using combined results of 2 modalities
(169, 2) (169, 2) (169,)
169 data are predicted using combined results of 2 modalities
(218, 2) (218, 2) (218,)
218 data are predicted using combined results of 2 modalities
(181, 2) (181, 2) (181,)
181 data are predicted using combined results of 2 modalities
(276, 2) (276, 2) (276,)
276 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(185, 2) (185, 2) (185,)
185 data are predicted using combined results of 2 modalities
(230, 2) (230, 2) (230,)
230 data are predicted using combined results of 2 modalities
(275, 2) (275, 2) (275,)
275 data are predicted using combined results of 2 modalities
(267, 2) (267, 2) (267,)
267 data are predicted using combined results of 2 modalities
(254, 2) (254, 2) (254,)
254 data are predi